In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
from oneibl import one
from export_funs import trialinfo_to_df
%matplotlib inline
one = one.ONE(offline=True)

SMALL_SIZE = 16
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

session_id = '259927fd-7563-4b03-bc5d-17b4d0fa7a55'
regdata = np.load('./fits/ZM_2240/2020-01-22_session_2020-07-13_probe0_pyfit_sklearn.p', allow_pickle=True)
trialsdf = trialinfo_to_df(session_id, wheel=False)
fits = regdata['fitobj']
weights = regdata['weights']
spk_times = one.load(session_id, dataset_types=['spikes.times'])[0]
spk_clu = one.load(session_id, dataset_types=['spikes.clusters'])[0]

In [25]:
import ipywidgets as iw
from brainbox.plot import peri_event_time_histogram
cells = weights['stimonL'].index.to_list()
kernels = list(weights.keys())
scores = fits.score()

@iw.interact
def plot_fits(cell=cells, kernel=kernels):
    if kernel[:-1] == 'stimon':
        side = 'contrastLeft' if kernel[-1] == 'L' else 'contrastRight'
        times = trialsdf[np.isfinite(trialsdf[side])].stimOn_times.values
    elif kernel[-4:] == 'rect':
        fdbk = 1 if kernel == 'correct' else -1
        times = trialsdf[trialsdf['feedbackType'] == fdbk].feedback_times

    if kernel in ['stimonL', 'stimonR', 'correct', 'incorrect']:
        fig, ax = plt.subplots(2, 1, figsize=(8, 12))
        ax0 = ax[0]
        ax1 = ax[1]
        peri_event_time_histogram(spk_times, spk_clu, times, cell, t_before=0., t_after=0.6, bin_size=0.02, ax=ax[0],
                                  error_bars='sem')
    else:
        fig, ax1 = plt.subplots(1, 1, figsize=(6, 6)) 
    colors = sns.cubehelix_palette(20, start=2, rot=0, dark=0, light=.8)
    ax1.plot(weights[kernel].loc[cell])
    plt.title(f'D^2 score of {scores[cell]}')
    plt.tight_layout()

interactive(children=(Dropdown(description='cell', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 17, …

In [2]:
from ibl_pipeline import histology, subject
from ibl_pipeline.analyses import behavior
bwm_sess = histology.ProbeTrajectory * behavior.SessionTrainingStatus & {'insertion_data_source': 'Planned'}

In [3]:
bwm_sess

subject_uuid,session_start_time start time,"probe_idx probe insertion number (0 corresponds to probe00, 1 corresponds to probe01)",insertion_data_source type of trajectory,coordinate_system_name,probe_trajectory_uuid,"x (um) medio-lateral coordinate relative to Bregma, left negative","y (um) antero-posterior coordinate relative to Bregma, back negative","z (um) dorso-ventral coordinate relative to Bregma, ventral negative",phi (degrees)[-180 180] azimuth,theta (degrees)[0 180] polar angle,depth (um) insertion depth,roll (degrees) roll angle of the probe,trajectory_ts,training_status,good_enough_for_brainwide_map
0812d02d-d62b-416d-8b66-7956db34ee32,2020-07-21 17:04:39,0,Planned,None,721ff288-3f67-43b2-a747-e5945407a48f,493.0,-2250.0,69.0,0.0,17.0,6222.0,0.0,2020-09-23 20:20:06,ready4delay,1
0812d02d-d62b-416d-8b66-7956db34ee32,2020-07-21 17:04:39,1,Planned,None,94509fb3-4c74-4397-8572-a93c3c56493c,-1281.0,2000.0,-979.0,180.0,15.0,4000.0,0.0,2020-09-23 20:21:54,ready4delay,1
0812d02d-d62b-416d-8b66-7956db34ee32,2020-07-22 20:51:44,0,Planned,None,fd091516-bf9b-407e-8a9b-40b56ea972b9,493.0,-1750.0,21.0,0.0,17.0,6272.0,0.0,2020-09-23 20:47:39,ready4delay,1
0812d02d-d62b-416d-8b66-7956db34ee32,2020-07-22 20:51:44,1,Planned,None,17ba1b34-b32d-4779-8c63-2dd5a54641d8,-800.0,2500.0,-1098.0,180.0,15.0,4000.0,0.0,2020-09-23 20:44:52,ready4delay,1
0812d02d-d62b-416d-8b66-7956db34ee32,2020-07-23 18:59:30,0,Planned,None,cf32290f-579a-408c-839d-6980ed3ee45d,-813.0,1750.0,-836.0,0.0,10.0,5558.0,0.0,2020-09-23 21:02:11,ready4delay,1
0812d02d-d62b-416d-8b66-7956db34ee32,2020-07-29 14:27:33,0,Planned,None,f01f78d3-74bf-465e-bec4-a81c68d8cbd9,-2355.0,-750.0,-408.0,0.0,10.0,6331.0,0.0,2020-09-23 21:07:14,ready4recording,0
0812d02d-d62b-416d-8b66-7956db34ee32,2020-07-30 19:57:05,0,Planned,None,09c16a6e-f752-4a40-8840-0e1bd699b55f,-2355.0,-1250.0,-313.0,0.0,10.0,6573.0,0.0,2020-09-23 21:09:52,ready4recording,0
082fa374-a10c-4c47-9c50-1c23e7490e30,2020-07-29 17:28:21,0,Planned,None,bbf647ac-c2b2-4db0-91ad-a1eeee18813c,-813.0,1750.0,-836.0,0.0,10.0,5558.0,0.0,2020-08-31 16:42:31,trained_1b,0
082fa374-a10c-4c47-9c50-1c23e7490e30,2020-07-29 17:28:21,1,Planned,None,8914506f-644a-4fdb-ad70-3772d78dd42d,-2243.0,-2000.0,-169.0,180.0,15.0,4000.0,0.0,2020-08-31 16:44:29,trained_1b,0
082fa374-a10c-4c47-9c50-1c23e7490e30,2020-07-30 17:47:20,0,Planned,None,dab87afc-df4e-4b59-90e7-cf04d0863f71,1281.0,2500.0,-1169.0,0.0,15.0,4000.0,0.0,2020-08-31 16:38:04,trained_1b,1
